# Virtual Sensing - Decision Tree Modelling

In [109]:
# libraries
import datetime
from datetime import date, timedelta
from os import path
import pandas as pd
import numpy as np
import statistics
import boto3
import s3fs
import sys
import re
import xgboost as xgb
import seaborn as sns
import statistics
from fastparquet import ParquetFile


import matplotlib.pyplot as plt
import plotly.offline as py
%matplotlib inline
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont


from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn import utils

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)

In [110]:
# constants
sys.path.append("../HistoricalData/")
from getData import get_data

UP_LEFT = (38.008050, -122.536985)    
UP_RIGHT = (38.008050, -122.186437)   
DOWN_RIGHT = (37.701933, -122.186437) 
DOWN_LEFT = (37.701933, -122.536985)  
START_DATE = '2019/09/01' 
END_DATE = '2019/09/30'   
START_HOUR = '0'        
END_HOUR = '24'  
# load data into dataframe
data_df = get_data(UP_LEFT, UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, START_DATE, END_DATE, START_HOUR, END_HOUR, 'Monthly')

In [111]:
data_df.columns

Index(['0_3um', '0_5um', '1_0um', '2_5um', '5_0um', '10_0um', 'pm1_0',
       'pm10_0', 'created', 'pm1_0_atm', 'pm2_5_atm', 'pm10_0_atm', 'uptime',
       'rssi', 'temperature', 'humidity', 'pm2_5_cf_1', 'device_loc_typ',
       'is_owner', 'sensor_id', 'sensor_name', 'parent_id', 'lat', 'lon',
       'thingspeak_primary_id', 'thingspeak_primary_id_read_key',
       'thingspeak_secondary_id', 'thingspeak_secondary_id_read_key', 'a_h',
       'high_reading_flag', 'hidden', 'city', 'county', 'zipcode',
       'created_at', 'year', 'month', 'day', 'hour', 'minute', 'wban_number',
       'call_sign', 'call_sign2', 'interval', 'call_sign3', 'zulu_time',
       'report_modifier', 'wind_data', 'wind_direction', 'wind_speed', 'gusts',
       'gust_speed', 'variable_winds', 'variable_wind_info', 'sys_maint_reqd',
       'agency_name', 'aqi', 'category', 'epa_pm25_unit', 'epa_pm25_value',
       'full_aqs_code', 'intl_aqs_code', 'raw_concentration', 'site_name',
       'wkday', 'daytype', 'time

In [114]:
# should be (726292, 68)
data_df.shape

(726292, 68)

Some data gets dropped because floating point doesn't quite match on sensor '23927' 

In [116]:
# winnow down the features
columns_to_keep = ['created', 'lat', 'lon', 'wind_data', 'wind_direction', 'wind_speed', 'gusts', 'gust_speed', 
                   'variable_winds', 'variable_wind_info', 'epa_pm25_value', 'wkday', 
                   'daytype', 'timeofday', 'wind_compass', 'temperature', 'humidity', 'hour', 'month']
X_data_df = data_df[columns_to_keep]
y_data_df = data_df['2_5um']

In [117]:
# fix datatyping
X_data_df.dtypes

created                 int64
lat                   float64
lon                   float64
wind_data             float32
wind_direction         object
wind_speed            float64
gusts                 float32
gust_speed            float64
variable_winds        float32
variable_wind_info     object
epa_pm25_value        float64
wkday                  object
daytype                object
timeofday              object
wind_compass           object
temperature           float64
humidity              float64
hour                   object
month                  object
dtype: object

In [118]:
X_data_df.wind_data = X_data_df.wind_data.astype(bool)
X_data_df.variable_winds = X_data_df.variable_winds.astype(bool)
X_data_df.gusts = X_data_df.gusts.astype(bool)
X_data_df.daytype = X_data_df.daytype.astype(str)
X_data_df.daytype = X_data_df.daytype.astype('category')
X_data_df.timeofday = X_data_df.timeofday.astype(str)
X_data_df.timeofday = X_data_df.timeofday.astype('category')
X_data_df.wind_compass = X_data_df.wind_compass.astype(str)
X_data_df.wind_compass = X_data_df.wind_compass.astype('category')
X_data_df.wkday = pd.to_numeric(X_data_df.wkday)
X_data_df.wkday = X_data_df.wkday.astype('category')
X_data_df.hour = X_data_df.hour.astype(int)
X_data_df.month = X_data_df.month.astype(int)

In [119]:
# handle variable winds missing values 
vrb_wind_range_readings = 0
mid_ranges = list()

for row in range(len(X_data_df)):
    if X_data_df.variable_winds.iloc[row]:
        info = X_data_df.variable_wind_info.iloc[row]
        if info:
            vrb_wind_range_readings += 1
            first, second = info.split('V')
            mid_range = int((int(first) + int(second)) / 2)
            if (X_data_df.wind_direction.iloc[row] == 'VRB'):
                X_data_df.wind_direction.lloc[row] = mid_range
            mid_ranges.append(mid_range)

replacement = statistics.mode(mid_ranges)
X_data_df = X_data_df.replace('VRB', replacement) # give variable wind the most frequent midpoint variable range
X_data_df = X_data_df.drop(columns = ['variable_wind_info'])

In [120]:
# handle missing wind values with means 
wind_direction_obs = pd.to_numeric(X_data_df.wind_direction.dropna())
wind_direction_avg = wind_direction_obs.mean()
wind_direction_avg = int(wind_direction_avg)
X_data_df.wind_direction = X_data_df.wind_direction.replace(np.nan, wind_direction_avg) # give missing wind direction the mean
X_data_df.wind_direction = X_data_df.wind_direction.replace('', wind_direction_avg)
X_data_df.wind_direction = X_data_df.wind_direction.astype(int)
wind_speed_obs = pd.to_numeric(X_data_df.wind_speed.dropna())
wind_speed_avg = wind_speed_obs.mean()
X_data_df.wind_speed = X_data_df.wind_speed.replace(np.nan, wind_speed_avg) # give missing wind speed the mean
X_data_df.wind_speed = X_data_df.wind_speed.replace('', wind_speed_avg)
X_data_df.gust_speed = X_data_df.gust_speed.replace(np.nan, 0)
X_data_df.gust_speed = X_data_df.gust_speed.replace('', 0)
X_data_df.epa_pm25_value = X_data_df.epa_pm25_value.replace(np.nan, X_data_df.epa_pm25_value.mean())
X_data_df.temperature = X_data_df.temperature.replace(np.nan, X_data_df.temperature.mean())
X_data_df.humidity = X_data_df.humidity.replace(np.nan, X_data_df.humidity.mean())

In [121]:
# one hot encode the categoricals
X_data_df = pd.concat([X_data_df,pd.get_dummies(X_data_df['timeofday'], prefix='timeofday')],axis=1)
X_data_df = pd.concat([X_data_df,pd.get_dummies(X_data_df['daytype'], prefix='daytype')], axis=1)
X_data_df = pd.concat([X_data_df,pd.get_dummies(X_data_df['wind_compass'], prefix='compass')], axis=1)

In [122]:
# remove the original columns that got one hot encoded, type the one hots as bools
X_data_df = X_data_df.drop(columns=['timeofday','daytype','wind_compass'])
X_data_df.timeofday_afternoon = X_data_df.timeofday_afternoon.astype(bool)
X_data_df.timeofday_evening = X_data_df.timeofday_evening.astype(bool)
X_data_df.timeofday_morning = X_data_df.timeofday_morning.astype(bool)
X_data_df.timeofday_night = X_data_df.timeofday_night.astype(bool)
X_data_df.daytype_Weekday = X_data_df.daytype_Weekday.astype(bool)
X_data_df.daytype_Weekend = X_data_df.daytype_Weekend.astype(bool)
X_data_df.compass_ERROR = X_data_df.compass_ERROR.astype(bool)
X_data_df.compass_East = X_data_df.compass_East.astype(bool)
X_data_df.compass_Missing = X_data_df.compass_Missing.astype(bool)
X_data_df['compass_No wind'] = X_data_df['compass_No wind'].astype(bool)
X_data_df.compass_North = X_data_df.compass_North.astype(bool)
X_data_df.compass_South = X_data_df.compass_South.astype(bool)
X_data_df.compass_West = X_data_df.compass_West.astype(bool)

In [123]:
# let's see again where we are at
X_data_df.dtypes

created                  int64
lat                    float64
lon                    float64
wind_data                 bool
wind_direction           int32
wind_speed             float64
gusts                     bool
gust_speed             float64
variable_winds            bool
epa_pm25_value         float64
wkday                    int64
temperature            float64
humidity               float64
hour                     int64
month                    int64
timeofday_afternoon       bool
timeofday_evening         bool
timeofday_morning         bool
timeofday_night           bool
daytype_Weekday           bool
daytype_Weekend           bool
compass_ERROR             bool
compass_East              bool
compass_Missing           bool
compass_No wind           bool
compass_North             bool
compass_South             bool
compass_West              bool
dtype: object

In [316]:
# confirm no NAs at this point
for col in X_data_df.columns:
    testy = X_data_df[col]
    print(col, testy.isna().sum())

created 0
lat 0
lon 0
wind_data 0
wind_direction 0
wind_speed 0
gusts 0
gust_speed 0
variable_winds 0
epa_pm25_value 0
wkday 0
temperature 0
humidity 0
hour 0
month 0
timeofday_afternoon 0
timeofday_evening 0
timeofday_morning 0
timeofday_night 0
daytype_Weekday 0
daytype_Weekend 0
compass_ERROR 0
compass_East 0
compass_Missing 0
compass_No wind 0
compass_North 0
compass_South 0
compass_West 0


# Decision Tree Modelling 

Decision Tree Modelling works by finding the best split for each node of the tree. The sklearn library implements Gini Impurity

In [414]:
columns_to_keep = ['created', 'lat', 'lon', 'epa_pm25_value', 'wkday', 'temperature', 'humidity', 'compass_East', 'compass_North', 'compass_South', 'compass_West', 'compass_No wind']  #'wind_direction', 'wind_speed', 
Xex_data_df = X_data_df[columns_to_keep]

In [415]:
# standardize the data
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler.fit(Xex_data_df)
Xex_data_df = scaler.transform(Xex_data_df)

In [416]:
# split data into train-dev-test
X_train_and_dev, X_test, y_train_and_dev, y_test = train_test_split(Xex_data_df, y_data_df, test_size=0.20, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_and_dev, y_train_and_dev, test_size=0.125, random_state=42)

Gini Impurity measures the disorder of a set of elements. It is calculated as the probability of mislabelling an element assuming that the element is randomly labelled according the the distribution of all the classes in the set. Decision Trees will try to find the split which decreases Gini Impurity the most across the two resulting nodes.

In [417]:
y_train = y_train.astype('int')
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y_train)
print(training_scores_encoded)
print(utils.multiclass.type_of_target(y_train))
# print(utils.multiclass.type_of_target(y_train.astype('int')))
# print(utils.multiclass.type_of_target(training_scores_encoded))

[3 1 3 ... 5 0 4]
multiclass


In [418]:
decision_tree = tree.DecisionTreeClassifier(max_depth = 3)

In [419]:
decision_tree.fit(X_train, training_scores_encoded)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [420]:
y_pred = decision_tree.predict(X_dev)

In [421]:
y_pred

array([2, 1, 1, ..., 2, 1, 0], dtype=int64)

In [422]:
# make any negative predictions 0
for row in range(len(y_pred)):
    if y_pred[row] < 0:
        y_pred[row] = 0

In [423]:
# calculate accuracy
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_dev, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance (R2) score: %.2f' % r2_score(y_dev, y_pred))

Coefficients: 
 [ 2.32693517e-01 -8.84236873e-02 -8.14402751e-01 -4.24428693e-02
 -6.78510905e-02  9.81955202e-01  9.27485784e-02 -2.47863792e-01
  3.85322461e-02  1.69695793e+01  2.17696200e-01 -1.58197524e+00
  2.43108763e+00 -5.50730067e-01 -1.87284822e-01  1.61287722e-01
  8.24721349e+11  8.39422304e+11  1.60188955e+12  1.66568873e+12
  2.28796713e+10  2.28796713e+10  5.59074923e+11  1.41803747e+11
  6.68510294e+11  3.17989627e+11  1.21163181e+11  1.53901690e+11
  4.61642051e+11]
Mean squared error: 228.06
Variance (R2) score: -0.03


In [424]:
for row in range(len(y_dev)):
    print("Predicted reading", y_pred[row], ". Actual reading", y_dev.iloc[row])

Predicted reading 2 . Actual reading 3.54
Predicted reading 1 . Actual reading 2.25
Predicted reading 1 . Actual reading 0.35
Predicted reading 1 . Actual reading 1.61
Predicted reading 2 . Actual reading 5.07
Predicted reading 2 . Actual reading 10.09
Predicted reading 2 . Actual reading 10.35
Predicted reading 2 . Actual reading 3.71
Predicted reading 2 . Actual reading 9.13
Predicted reading 2 . Actual reading 0.51
Predicted reading 2 . Actual reading 1.13
Predicted reading 1 . Actual reading 1.82
Predicted reading 2 . Actual reading 4.27
Predicted reading 1 . Actual reading 2.99
Predicted reading 1 . Actual reading 2.21
Predicted reading 2 . Actual reading 3.17
Predicted reading 2 . Actual reading 6.76
Predicted reading 2 . Actual reading 1.47
Predicted reading 2 . Actual reading 2.41
Predicted reading 1 . Actual reading 2.31
Predicted reading 0 . Actual reading 0.79
Predicted reading 2 . Actual reading 0.24
Predicted reading 0 . Actual reading 5.31
Predicted reading 1 . Actual rea

Predicted reading 1 . Actual reading 1.57
Predicted reading 1 . Actual reading 1.53
Predicted reading 1 . Actual reading 0.95
Predicted reading 2 . Actual reading 4.68
Predicted reading 2 . Actual reading 1.49
Predicted reading 2 . Actual reading 8.15
Predicted reading 1 . Actual reading 1.41
Predicted reading 1 . Actual reading 4.94
Predicted reading 1 . Actual reading 4.51
Predicted reading 2 . Actual reading 1.62
Predicted reading 2 . Actual reading 3.6
Predicted reading 2 . Actual reading 4.57
Predicted reading 0 . Actual reading 3.31
Predicted reading 1 . Actual reading 1.07
Predicted reading 2 . Actual reading 3.38
Predicted reading 2 . Actual reading 2.97
Predicted reading 2 . Actual reading 8.58
Predicted reading 1 . Actual reading 0.65
Predicted reading 2 . Actual reading 6.24
Predicted reading 2 . Actual reading 1.61
Predicted reading 0 . Actual reading 1.47
Predicted reading 0 . Actual reading 2.15
Predicted reading 2 . Actual reading 2.26
Predicted reading 2 . Actual readin

Predicted reading 2 . Actual reading 1.68
Predicted reading 1 . Actual reading 0.73
Predicted reading 2 . Actual reading 7.42
Predicted reading 1 . Actual reading 3.69
Predicted reading 2 . Actual reading 3.45
Predicted reading 2 . Actual reading 2.79
Predicted reading 0 . Actual reading 0.49
Predicted reading 2 . Actual reading 4.01
Predicted reading 0 . Actual reading 6.43
Predicted reading 2 . Actual reading 3.55
Predicted reading 2 . Actual reading 5.5
Predicted reading 0 . Actual reading 0.0
Predicted reading 0 . Actual reading 4.55
Predicted reading 2 . Actual reading 7.06
Predicted reading 2 . Actual reading 3.42
Predicted reading 1 . Actual reading 10.45
Predicted reading 2 . Actual reading 14.23
Predicted reading 2 . Actual reading 2.67
Predicted reading 2 . Actual reading 2.85
Predicted reading 2 . Actual reading 1.47
Predicted reading 1 . Actual reading 0.0
Predicted reading 2 . Actual reading 4.67
Predicted reading 1 . Actual reading 5.31
Predicted reading 2 . Actual readin

Predicted reading 1 . Actual reading 0.99
Predicted reading 1 . Actual reading 1.13
Predicted reading 1 . Actual reading 0.1
Predicted reading 2 . Actual reading 7.24
Predicted reading 2 . Actual reading 4.24
Predicted reading 2 . Actual reading 6.03
Predicted reading 1 . Actual reading 8.66
Predicted reading 0 . Actual reading 0.09
Predicted reading 0 . Actual reading 0.25
Predicted reading 2 . Actual reading 4.5
Predicted reading 2 . Actual reading 1.65
Predicted reading 2 . Actual reading 4.26
Predicted reading 2 . Actual reading 6.79
Predicted reading 1 . Actual reading 5.67
Predicted reading 1 . Actual reading 8.74
Predicted reading 1 . Actual reading 1.19
Predicted reading 1 . Actual reading 1.7
Predicted reading 1 . Actual reading 4.07
Predicted reading 1 . Actual reading 3.77
Predicted reading 1 . Actual reading 0.63
Predicted reading 1 . Actual reading 10.0
Predicted reading 1 . Actual reading 13.42
Predicted reading 0 . Actual reading 1.75
Predicted reading 1 . Actual reading

Predicted reading 1 . Actual reading 1.01
Predicted reading 2 . Actual reading 6.56
Predicted reading 2 . Actual reading 3.34
Predicted reading 2 . Actual reading 1.83
Predicted reading 1 . Actual reading 71.04
Predicted reading 2 . Actual reading 16.57
Predicted reading 2 . Actual reading 3.5
Predicted reading 0 . Actual reading 0.47
Predicted reading 2 . Actual reading 4.54
Predicted reading 1 . Actual reading 0.0
Predicted reading 0 . Actual reading 5.53
Predicted reading 2 . Actual reading 7.01
Predicted reading 1 . Actual reading 2.75
Predicted reading 2 . Actual reading 3.41
Predicted reading 1 . Actual reading 0.39
Predicted reading 0 . Actual reading 4.32
Predicted reading 1 . Actual reading 1.32
Predicted reading 2 . Actual reading 3.03
Predicted reading 2 . Actual reading 0.69
Predicted reading 0 . Actual reading 1.45
Predicted reading 1 . Actual reading 2.0
Predicted reading 1 . Actual reading 1.03
Predicted reading 2 . Actual reading 4.55
Predicted reading 0 . Actual readin

...

Predicted reading 1 . Actual reading 4.9
Predicted reading 2 . Actual reading 1.65
Predicted reading 1 . Actual reading 2.64
Predicted reading 1 . Actual reading 2.23
Predicted reading 1 . Actual reading 2.03
Predicted reading 1 . Actual reading 5.06
Predicted reading 1 . Actual reading 1.88
Predicted reading 2 . Actual reading 0.64
Predicted reading 2 . Actual reading 1.19
Predicted reading 2 . Actual reading 5.1
Predicted reading 1 . Actual reading 1.29
Predicted reading 2 . Actual reading 0.81
Predicted reading 2 . Actual reading 1.26
Predicted reading 2 . Actual reading 0.97
Predicted reading 0 . Actual reading 2.03
Predicted reading 2 . Actual reading 4.35
Predicted reading 1 . Actual reading 0.46
Predicted reading 1 . Actual reading 1.36
Predicted reading 1 . Actual reading 0.58
Predicted reading 2 . Actual reading 2.37
Predicted reading 2 . Actual reading 4.69
Predicted reading 2 . Actual reading 5.36
Predicted reading 1 . Actual reading 0.24
Predicted reading 1 . Actual reading

Predicted reading 2 . Actual reading 17.62
Predicted reading 1 . Actual reading 1.56
Predicted reading 2 . Actual reading 0.51
Predicted reading 1 . Actual reading 3.25
Predicted reading 0 . Actual reading 2.62
Predicted reading 1 . Actual reading 1.22
Predicted reading 2 . Actual reading 10.42
Predicted reading 0 . Actual reading 2.0
Predicted reading 1 . Actual reading 1.12
Predicted reading 2 . Actual reading 2.34
Predicted reading 0 . Actual reading 4.78
Predicted reading 1 . Actual reading 3.09
Predicted reading 1 . Actual reading 3.16
Predicted reading 2 . Actual reading 4.62
Predicted reading 2 . Actual reading 3.74
Predicted reading 2 . Actual reading 2.07
Predicted reading 1 . Actual reading 2.91
Predicted reading 2 . Actual reading 4.19
Predicted reading 0 . Actual reading 2.58
Predicted reading 2 . Actual reading 12.27
Predicted reading 1 . Actual reading 2.87
Predicted reading 1 . Actual reading 3.75
Predicted reading 2 . Actual reading 4.43
Predicted reading 2 . Actual rea

Predicted reading 1 . Actual reading 3.51
Predicted reading 2 . Actual reading 4.01
Predicted reading 2 . Actual reading 1.35
Predicted reading 1 . Actual reading 1.68
Predicted reading 1 . Actual reading 1.2
Predicted reading 2 . Actual reading 3.16
Predicted reading 0 . Actual reading 0.0
Predicted reading 2 . Actual reading 7.97
Predicted reading 2 . Actual reading 3.56
Predicted reading 1 . Actual reading 3.06
Predicted reading 0 . Actual reading 0.86
Predicted reading 1 . Actual reading 5.62
Predicted reading 2 . Actual reading 2.12
Predicted reading 2 . Actual reading 4.57
Predicted reading 0 . Actual reading 2.0
Predicted reading 1 . Actual reading 5.47
Predicted reading 2 . Actual reading 2.21
Predicted reading 2 . Actual reading 3.85
Predicted reading 1 . Actual reading 1.48
Predicted reading 1 . Actual reading 7.04
Predicted reading 2 . Actual reading 2.89
Predicted reading 1 . Actual reading 3.52
Predicted reading 0 . Actual reading 4.07
Predicted reading 1 . Actual reading 

Predicted reading 2 . Actual reading 2.06
Predicted reading 2 . Actual reading 9.37
Predicted reading 1 . Actual reading 0.0
Predicted reading 1 . Actual reading 1.23
Predicted reading 1 . Actual reading 3.07
Predicted reading 1 . Actual reading 3.27
Predicted reading 0 . Actual reading 2.3
Predicted reading 1 . Actual reading 5.01
Predicted reading 2 . Actual reading 1.43
Predicted reading 1 . Actual reading 5.87
Predicted reading 1 . Actual reading 3.51
Predicted reading 2 . Actual reading 5.07
Predicted reading 2 . Actual reading 1.24
Predicted reading 2 . Actual reading 5.49
Predicted reading 2 . Actual reading 6.97
Predicted reading 2 . Actual reading 2.87
Predicted reading 2 . Actual reading 5.29
Predicted reading 1 . Actual reading 17.69
Predicted reading 2 . Actual reading 2.71
Predicted reading 0 . Actual reading 5.09
Predicted reading 1 . Actual reading 1.74
Predicted reading 2 . Actual reading 4.83
Predicted reading 1 . Actual reading 4.7
Predicted reading 2 . Actual reading

Predicted reading 2 . Actual reading 13.25
Predicted reading 2 . Actual reading 2.82
Predicted reading 1 . Actual reading 5.1
Predicted reading 1 . Actual reading 0.75
Predicted reading 1 . Actual reading 0.32
Predicted reading 1 . Actual reading 6.47
Predicted reading 1 . Actual reading 1.78
Predicted reading 2 . Actual reading 6.61
Predicted reading 1 . Actual reading 0.49
Predicted reading 1 . Actual reading 2.44
Predicted reading 2 . Actual reading 2.87
Predicted reading 2 . Actual reading 4.29
Predicted reading 0 . Actual reading 0.47
Predicted reading 1 . Actual reading 0.0
Predicted reading 1 . Actual reading 2.59
Predicted reading 1 . Actual reading 2.36
Predicted reading 0 . Actual reading 0.56
Predicted reading 2 . Actual reading 7.09
Predicted reading 2 . Actual reading 24.71
Predicted reading 0 . Actual reading 5.04
Predicted reading 2 . Actual reading 2.66
Predicted reading 1 . Actual reading 4.0
Predicted reading 2 . Actual reading 43.26
Predicted reading 1 . Actual readi